In [1]:
from pathlib import Path
from tqdm import tqdm
from virtualizarr import open_virtual_dataset, open_virtual_mfdataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry
from icechunk.xarray import to_icechunk
from dask.diagnostics import ProgressBar

import obstore as obs
import xarray as xr
import icechunk
import hvplot.xarray
import numpy as np

import warnings
warnings.filterwarnings(
  "ignore",
  message="Numcodecs codecs are not in the Zarr version 3 specification*",
  category=UserWarning
)
icechunk.set_logs_filter(
    "icechunk::storage::object_store=error"
)  # remove the warning due to the local icechunk store

In [2]:
ROOT_DIR = Path("/Volumes/T7")
RAW_FILES_DIR = ROOT_DIR / "raw_files/hackathon-meteo-france"

BUCKETS_DIR = ROOT_DIR / "buckets"
ICECHUNK_BUCKET = BUCKETS_DIR / "hackathon-meteo-france"
PREFIX_RCM = "RCM"

prefix = "file:///Volumes/"

# Get file paths

In [3]:
variables = [
    # "hussAdjust",
    "prAdjust",
    # "rldsAdjust",
    # "rsdsAdjust",
    # "sfcWindAdjust",
    # "tasAdjust",
    # "tasmaxAdjust",
    # "tasminAdjust",
]

In [4]:
all_files = list((RAW_FILES_DIR / "RCM").rglob(f"**/historical/**/*.nc"))

files = {
    k: list((RAW_FILES_DIR / "RCM").rglob(f"**/historical/**/{k}*.nc"))
    for k in variables
}

# assert len(all_files) == sum([len(v) for v in files.values()])

{k: len(v) for k,v in files.items()}

{'prAdjust': 17}

In [5]:
files["prAdjust"][:3]

[PosixPath('/Volumes/T7/raw_files/hackathon-meteo-france/RCM/EURO-CORDEX/EUR-12/EC-Earth3/r1i1p1f1/RACMO23E/historical/day/prAdjust/version-hackathon-102025/prAdjust_FR-Metro_EC-Earth3_historical_r1i1p1f1_KNMI_RACMO23E_v1-r1_MF-CDFt-SAFRAN-1985-2014_day_19500101-20141231.nc'),
 PosixPath('/Volumes/T7/raw_files/hackathon-meteo-france/RCM/EURO-CORDEX/EUR-12/IPSL-CM6A-LR/r1i1p1f1/HCLIM43-ALADIN/historical/day/prAdjust/version-hackathon-102025/prAdjust_FR-Metro_IPSL-CM6A-LR_historical_r1i1p1f1_HCLIMcom-DMI_HCLIM43-ALADIN_v1-r1_MF-CDFt-SAFRAN-1985-2014_day_19500101-20141231.nc'),
 PosixPath('/Volumes/T7/raw_files/hackathon-meteo-france/RCM/EURO-CORDEX/EUR-12/MPI-ESM1-2-HR/r1i1p1f1/ICON-CLM-202407-1-1/historical/day/prAdjust/version-hackathon-102025/prAdjust_FR-Metro_MPI-ESM1-2-HR_historical_r1i1p1f1_CLMcom-DWD_ICON-CLM-202407-1-1_v1-r1_MF-CDFt-SAFRAN-1985-2014_day_19500101-20141231.nc')]

# Load data

In [6]:
def process_ds(ds, var_name: str):
    ds = ds.drop_vars(["lat_bnds", "lon_bnds"])
    
    dims = [
        "input_driving_source_id",
        "input_driving_variant_label",
        "input_source_id",
    ]
    for d in dims:
        ds = ds.expand_dims({d: np.array([ds.attrs[d]], dtype=str)})

    # restrict to >1950, since some models are missing 1950
    ds = ds.sel(time=slice("1951", "2014"))

    # create a unique dimension
    # 1. create a multi-index
    ds = ds.stack(source_id=dims)
    # 2. flatten the multi-index
    source_id_index = ds.indexes["source_id"].map(lambda x: f"{x[0]}:{x[1]}:{x[2]}")
    ds = ds.assign_coords(source_id_bis=("source_id", source_id_index))
    # 3. and only keep the flattened index
    ds = ds.set_index(source_id="source_id_bis")

    # drop bnds dim and associated time_bnds variable
    ds = ds.drop_dims("bnds")

    # ensure this order
    ds = ds.transpose("source_id", "time", "y", "x")
    return ds

In [8]:
f = files["prAdjust"][0]
var_name = "prAdjust"
ds = xr.open_dataset(f, chunks={})
ds = process_ds(ds, var_name)

In [9]:
ds

<xarray.Dataset> Size: 2GB
Dimensions:                      (source_id: 1, time: 23376, y: 134, x: 143)
Coordinates:
  * source_id                    (source_id) object 8B 'EC-Earth3:r1i1p1f1:RA...
  * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
  * y                            (y) float64 1kB 1.617e+06 ... 2.681e+06
  * x                            (x) float64 1kB 6e+04 6.8e+04 ... 1.196e+06
    lon                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
    lat                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
    input_source_id              (source_id) object 8B 'RACMO23E'
    input_driving_source_id      (source_id) object 8B 'EC-Earth3'
    input_driving_variant_label  (source_id) object 8B 'r1i1p1f1'
Data variables:
    LambertParisII               (source_id) int32 4B 1
    prAdjust                     (source_id, time, y, x) float32 2GB dask.array<chunksize=(1, 128, 134, 143), meta=np.ndarray>
Attributes: (12/71)
    title:                           RACMO23E model output corrected over met...
    summary:                         Data were post-processed by the Météo-Fr...
    id:                              hdl:21.14103/79b2edf7-a707-45eb-af94-aa6...
    Conventions:                     CF-1.11 ACDD-1.3
    standard_name_vocabulary:        CF Standard Name Table v81
    input_domain:                    Europe
    ...                              ...
    date_created:                    2025-11-13T13:30:29.072166
    product:                         bias-adjusted output
    license:                         CORDEX model data produced by KNMI is li...
    product_version:                 v1
    comment:                         Preliminary data released in the Fall 20...
    keywords:                        precipitation, climate, France, total pr...

In [10]:
datasets = {}

for var_name, urls in files.items():
    datasets[var_name] = []
    for f in tqdm(urls):
        ds = xr.open_dataset(f, chunks={})
        ds = process_ds(ds, var_name)
        datasets[var_name].append(ds)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:04<00:00,  3.64it/s]


In [13]:
[d.prAdjust.shape for d in datasets["prAdjust"]]

[(1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143),
 (1, 23376, 134, 143)]

In [14]:
x = xr.combine_nested(
    datasets["prAdjust"],
    concat_dim="source_id",
    combine_attrs='drop_conflicts',
)

In [32]:
x

<xarray.Dataset> Size: 30GB
Dimensions:                      (model_id: 17, time: 23376, y: 134, x: 143,
                                  bnds: 2)
Coordinates:
  * model_id                     (model_id) object 136B MultiIndex
  * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
  * y                            (y) float64 1kB 1.617e+06 ... 2.681e+06
  * x                            (x) float64 1kB 6e+04 6.8e+04 ... 1.196e+06
    lon                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
    lat                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
  * input_source_id              (model_id) object 136B 'RACMO23E' ... 'CNRM-...
  * input_driving_variant_label  (model_id) object 136B 'r1i1p1f1' ... 'r1i1p...
  * input_driving_source_id      (model_id) object 136B 'EC-Earth3' ... 'CMCC...
    model_id_bis                 (model_id) object 136B 'RACMO23E_r1i1p1f1_EC...
Dimensions without coordinates: bnds
Data variables:
    LambertParisII               (model_id) int32 68B 1 1 1 1 1 1 ... 1 1 1 1 1
    prAdjust                     (time, y, x, model_id) float32 30GB dask.array<chunksize=(19, 134, 143, 1), meta=np.ndarray>
    time_bnds                    (time, bnds, model_id) datetime64[ns] 6MB dask.array<chunksize=(19, 2, 1), meta=np.ndarray>
Attributes: (12/51)
    summary:                         Data were post-processed by the Météo-Fr...
    Conventions:                     CF-1.11 ACDD-1.3
    standard_name_vocabulary:        CF Standard Name Table v81
    input_domain:                    Europe
    input_domain_id:                 EUR-12
    project:                         Socle-FR-Metro-Climat-2025
    ...                              ...
    time_coverage_duration:          P1Y
    time_coverage_resolution:        P1D
    product:                         bias-adjusted output
    product_version:                 v1
    comment:                         Preliminary data released in the Fall 20...
    keywords:                        precipitation, climate, France, total pr...

In [11]:
x.model_id.values

array([('RACMO23E', 'r1i1p1f1', 'EC-Earth3'),
       ('HCLIM43-ALADIN', 'r1i1p1f1', 'IPSL-CM6A-LR'),
       ('ICON-CLM-202407-1-1', 'r1i1p1f1', 'MPI-ESM1-2-HR'),
       ('HCLIM43-ALADIN', 'r1i1p1f1', 'MPI-ESM1-2-HR'),
       ('RACMO23E', 'r1i1p1f1', 'MPI-ESM1-2-HR'),
       ('HCLIM43-ALADIN', 'r1i1p1f1', 'MIROC6'),
       ('CNRM-ALADIN64E1', 'r1i1p1f2', 'CNRM-ESM2-1'),
       ('HCLIM43-ALADIN', 'r1i1p1f2', 'CNRM-ESM2-1'),
       ('RACMO23E', 'r1i1p1f2', 'CNRM-ESM2-1'),
       ('CNRM-ALADIN64E1', 'r15i1p1f2', 'CNRM-ESM2-1'),
       ('CNRM-ALADIN64E1', 'r14i1p1f2', 'CNRM-ESM2-1'),
       ('CNRM-ALADIN64E1', 'r1i1p1f1', 'NorESM2-MM'),
       ('HCLIM43-ALADIN', 'r1i1p1f1', 'NorESM2-MM'),
       ('RACMO23E', 'r1i1p1f1', 'NorESM2-MM'),
       ('HCLIM43-ALADIN', 'r1i1p1f1', 'EC-Earth3-Veg'),
       ('RACMO23E', 'r1i1p1f1', 'EC-Earth3-Veg'),
       ('CNRM-ALADIN64E1', 'r1i1p1f1', 'CMCC-CM2-SR5')], dtype=object)

In [14]:
x

<xarray.Dataset> Size: 30GB
Dimensions:                      (model_id: 17, time: 23376, y: 134, x: 143,
                                  bnds: 2)
Coordinates:
  * model_id                     (model_id) object 136B MultiIndex
  * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
  * y                            (y) float64 1kB 1.617e+06 ... 2.681e+06
  * x                            (x) float64 1kB 6e+04 6.8e+04 ... 1.196e+06
    lon                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
    lat                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
  * input_source_id              (model_id) object 136B 'RACMO23E' ... 'CNRM-...
  * input_driving_variant_label  (model_id) object 136B 'r1i1p1f1' ... 'r1i1p...
  * input_driving_source_id      (model_id) object 136B 'EC-Earth3' ... 'CMCC...
Dimensions without coordinates: bnds
Data variables:
    LambertParisII               (model_id) int32 68B 1 1 1 1 1 1 ... 1 1 1 1 1
    prAdjust                     (time, y, x, model_id) float32 30GB dask.array<chunksize=(19, 134, 143, 1), meta=np.ndarray>
    time_bnds                    (time, bnds, model_id) datetime64[ns] 6MB dask.array<chunksize=(19, 2, 1), meta=np.ndarray>
Attributes: (12/51)
    summary:                         Data were post-processed by the Météo-Fr...
    Conventions:                     CF-1.11 ACDD-1.3
    standard_name_vocabulary:        CF Standard Name Table v81
    input_domain:                    Europe
    input_domain_id:                 EUR-12
    project:                         Socle-FR-Metro-Climat-2025
    ...                              ...
    time_coverage_duration:          P1Y
    time_coverage_resolution:        P1D
    product:                         bias-adjusted output
    product_version:                 v1
    comment:                         Preliminary data released in the Fall 20...
    keywords:                        precipitation, climate, France, total pr...

# Save to zarr store

In [12]:
def save_dataset(ds, repo, path: str):
    session = repo.writable_session("main")
    with ProgressBar():
        to_icechunk(ds, session, group=path)
    msg = f"Import dataset {path}"
    snapshot = session.commit(msg)
    print(f"{snapshot} - {msg}")

In [13]:
storage = icechunk.local_filesystem_storage(
    # optional: we can add a prefix here
    path=ICECHUNK_BUCKET / PREFIX_RCM
)
config = icechunk.RepositoryConfig.default()

repo = icechunk.Repository.open_or_create(
    storage,
    config,
    authorize_virtual_chunk_access={prefix: None}
)

In [15]:
var_name="prAdjust"

In [16]:
save_dataset(x, repo, path=var_name)

NotImplementedError: variable 'model_id' is a MultiIndex, which cannot yet be serialized. Instead, either use reset_index() to convert MultiIndex levels into coordinate variables instead or use https://cf-xarray.readthedocs.io/en/latest/coding.html.

[########################################] | 100% Completed | 599.78 s


# Check

In [3]:
storage = icechunk.local_filesystem_storage(
    # optional: we can add a prefix here
    path=ICECHUNK_BUCKET / PREFIX_RCM
)
new_repo = icechunk.Repository.open(
    storage,
    authorize_virtual_chunk_access={prefix: None}
)
new_session = new_repo.readonly_session("main")

dt = xr.open_datatree(new_session.store, engine='zarr', zarr_format=3, chunks={})
dt

<xarray.DataTree>
Group: /
├── Group: /hussAdjust
│       Dimensions:                      (source_id: 17, time: 23376, y: 134, x: 143)
│       Coordinates:
│         * source_id                    (source_id) object 136B 'EC-Earth3:r1i1p1f1:...
│         * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
│         * y                            (y) float64 1kB 1.617e+06 ... 2.681e+06
│         * x                            (x) float64 1kB 6e+04 6.8e+04 ... 1.196e+06
│           input_driving_variant_label  (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           input_driving_source_id      (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           lon                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│           input_source_id              (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           lat                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│       Data variables:
│           LambertParisII               (source_id) int32 68B dask.array<chunksize=(1,), meta=np.ndarray>
│           hussAdjust                   (source_id, time, y, x) float32 30GB dask.array<chunksize=(1, 128, 134, 143), meta=np.ndarray>
│       Attributes: (12/51)
│           summary:                         Data were post-processed by the Météo-Fr...
│           Conventions:                     CF-1.11 ACDD-1.3
│           standard_name_vocabulary:        CF Standard Name Table v81
│           input_domain:                    Europe
│           input_domain_id:                 EUR-12
│           project:                         Socle-FR-Metro-Climat-2025
│           ...                              ...
│           time_coverage_duration:          P1Y
│           time_coverage_resolution:        P1D
│           product:                         bias-adjusted output
│           product_version:                 v1
│           comment:                         Preliminary data released in the Fall 20...
│           keywords:                        humidity, climate, France, near-surface ...
├── Group: /sfcWindAdjust
│       Dimensions:                      (source_id: 17, time: 23376, y: 134, x: 143)
│       Coordinates:
│         * source_id                    (source_id) object 136B 'EC-Earth3:r1i1p1f1:...
│         * time                         (time) datetime64[ns] 187kB 1951-01-01T12:00...
│         * y                            (y) float64 1kB 1.617e+06 ... 2.681e+06
│         * x                            (x) float64 1kB 6e+04 6.8e+04 ... 1.196e+06
│           input_driving_source_id      (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           lat                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│           input_driving_variant_label  (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           input_source_id              (source_id) object 136B dask.array<chunksize=(17,), meta=np.ndarray>
│           lon                          (y, x) float64 153kB dask.array<chunksize=(134, 143), meta=np.ndarray>
│       Data variables:
│           LambertParisII               (source_id) int32 68B dask.array<chunksize=(1,), meta=np.ndarray>
│           sfcWindAdjust                (source_id, time, y, x) float32 30GB dask.array<chunksize=(1, 128, 134, 143), meta=np.ndarray>
│       Attributes: (12/51)
│           summary:                         Data were post-processed by the Météo-Fr...
│           Conventions:                     CF-1.11 ACDD-1.3
│           standard_name_vocabulary:        CF Standard Name Table v81
│           input_domain:                    Europe
│           input_domain_id:                 EUR-12
│           project:                         Socle-FR-Metro-Climat-2025
│           ...                              ...
│           time_coverage_duration:   

In [8]:
var_name = "prAdjust"
dt[var_name][var_name].isel(time=100, source_id=3).hvplot()

:Image   [x,y]   (Bias Corrected Precipitation)